# 문제 생성 Agent 시연 성공

In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 1️⃣ 출력 스키마 정의
schemas = [
    ResponseSchema(name="question", description="한글 문제문"),
    ResponseSchema(name="choice",   description="보기 5개 (리스트)"),
]
parser = StructuredOutputParser.from_response_schemas(schemas)
fmt_instructions = parser.get_format_instructions()   # {"question": ..., "choice": ...}

# 2️⃣ 프롬프트
template = """
너는 고등학교 미적분 교재 집필진이다.

◆ 주제
{topics}

◆ 범위
{range}

◆ 범위 내용 요약 설명
{summarized}

◆ 난이도
{difficulty}

◆ 문제 예시
{quiz_examples}

---
아래 지시를 **반드시** 지켜라:
{format_instructions}
"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": fmt_instructions}
)

# 3️⃣ LLM (JSON 응답 강제)
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    response_format={"type": "json_object"}  # ← JSON 외 형식 차단
)

chain = prompt | llm | parser

result = chain.invoke({
    "topics": "함수의 극한 확인문제",
    "range": "2.2 The Limit of Functions",
    "summarized": "극한의 정의, 한쪽·무한 극한, 수직 점근선",
    "difficulty": "풀이 5줄 이내",
    "quiz_examples": "(예시 문제)"
})

print(result)
# {'question': '...', 'choice': ['A. ...', 'B. ...', ...]}


/Users/wooseok/Documents/4학년 1학기/종합설계프로젝트2/code/ai/test/wooseok/ema_ai/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3489: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if await self.run_code(code, result, async_=asy):


{'question': '다음 함수의 극한을 구하시오: lim(x→2) (3x^2 - 2x + 1)', 'choice': ['5', '7', '9', '11', '13']}
